In [1]:
import numpy as np
from pathlib import Path
import pandas as pd
from collections import defaultdict

In [2]:
secrets = np.array(
    [int(x) for x in Path("data/22.txt").read_text().strip().split("\n")]
)

In [3]:
def evolve(secrets):
    secrets = np.bitwise_xor(secrets * 64, secrets) % 16777216
    secrets = np.bitwise_xor(secrets // 32, secrets) % 16777216
    return np.bitwise_xor(secrets * 2048, secrets) % 16777216


def repeatedly_apply(f, x, n):
    for _ in range(n):
        x = f(x)
    return x


repeatedly_apply(evolve, secrets, 2000).sum()

np.int64(19241711734)

In [4]:
res = [secrets]
for i in range(1, 2001):
    secrets = evolve(secrets)
    res.append(secrets)

df = pd.DataFrame(res)
df.columns = res[0]

In [5]:
result = defaultdict(lambda: dict())


def f(s: pd.Series):
    for i in range(4, len(s)):
        key = ",".join(str(int(a)) for a in s.values[max(i - 3, 0) : i + 1] if a == a)
        if key not in result[s.name]:
            result[s.name][key] = df[s.name].iloc[i] % 10
    return 0


_ = (df % 10).diff().apply(f, axis=0)

In [6]:
s = pd.Series()
for v in result:
    s = s.add(pd.Series(result[v]), fill_value=0)

s.max()

2058.0